<h2 style='text-align:center;'>🖌️ Virtual Hand Gesture — Air Painter Suite</h2>

This module demonstrates three approaches to virtual painting using hand gestures: (1) pinch-based drawing math, (2) fingertip basic painter, and (3) full air-brush with toolbar.

<h2 style='text-align:center;'>📐 Pinch-to-Draw (Math)</h2>

Run `gesture_distance_draw.py` to draw by pinching (thumb+index).

In [ ]:
# from gesture_distance_draw import run_pinch_draw
# run_pinch_draw()

<h2 style='text-align:center;'>✍️ Basic Virtual Painter</h2>

Run `gesture_painter_basic.py` to use index fingertip as brush. Clear canvas by raising all fingers.

In [ ]:
# from gesture_painter_basic import run_basic_painter
# run_basic_painter()

<h2 style='text-align:center;'>🎨 Full Air Brush Painter</h2>

Run `air_brush_virtual_painter.py` for toolbar-driven color selection and save functionality.

In [ ]:
# from air_brush_virtual_painter import run_air_brush
# run_air_brush()

<h2 style='text-align:center;'>📜 Raw Scripts (Sir's originals)</h2>

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Create a blank canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Track finger state
drawing = False
prev_x, prev_y = 0, 0

def euclidean_distance(pt1, pt2):
    return math.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Get landmarks for index fingertip and thumb tip
            index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

            ix, iy = int(index_finger.x * w), int(index_finger.y * h)
            tx, ty = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Distance between thumb and index finger
            distance = euclidean_distance((ix, iy), (tx, ty))

            # Gesture math: if distance is small, pinch (draw mode)
            if distance < 40:
                drawing = True
                if prev_x == 0 and prev_y == 0:
                    prev_x, prev_y = ix, iy
                cv2.line(canvas, (prev_x, prev_y), (ix, iy), (255, 0, 0), 5)
                prev_x, prev_y = ix, iy
            else:
                drawing = False
                prev_x, prev_y = 0, 0

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Overlay canvas on frame
    frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Instructions
    cv2.putText(frame, 'Pinch (Thumb + Index) to Draw', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 2)

    cv2.imshow("Virtual Painter", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Hand Detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Webcam Capture
cap = cv2.VideoCapture(0)
canvas = None

# Drawing settings
draw_color = (255, 0, 255)  # Purple
brush_thickness = 8

# Previous point
xp, yp = 0, 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)  # Mirror the frame
    h, w, c = frame.shape

    if canvas is None:
        canvas = np.zeros_like(frame)

    # Convert to RGB for MediaPipe
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            lm_list = []
            for id, lm in enumerate(handLms.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append((cx, cy))

            if lm_list:
                x1, y1 = lm_list[8]  # Index finger tip
                x2, y2 = lm_list[12] # Middle finger tip

                # Check if only index finger is up
                fingers = []
                for tip_id in [8, 12, 16, 20]:
                    fingers.append(lm_list[tip_id][1] < lm_list[tip_id - 2][1])

                if fingers[0] and not fingers[1]:  # Drawing mode
                    cv2.circle(frame, (x1, y1), 8, draw_color, -1)
                    if xp == 0 and yp == 0:
                        xp, yp = x1, y1
                    cv2.line(canvas, (xp, yp), (x1, y1), draw_color, brush_thickness)
                    xp, yp = x1, y1
                else:
                    xp, yp = 0, 0

                # Clear canvas if all fingers up
                if all(fingers):
                    canvas = np.zeros_like(frame)

            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

    # Merge canvas and frame
    gray_canvas = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_canvas, 20, 255, cv2.THRESH_BINARY)
    inv_mask = cv2.bitwise_not(mask)
    frame_bg = cv2.bitwise_and(frame, frame, mask=inv_mask)
    frame_fg = cv2.bitwise_and(canvas, canvas, mask=mask)
    final = cv2.add(frame_bg, frame_fg)

    cv2.imshow("Virtual Painter", final)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time

# Initialize MediaPipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

# Variables
draw_color = (255, 0, 255)  # Default: Purple
brush_thickness = 7
eraser_thickness = 50
xp, yp = 0, 0
img_canvas = np.zeros((720, 1280, 3), np.uint8)

# Open Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Finger tips
tip_ids = [4, 8, 12, 16, 20]

def fingers_up(hand_landmarks):
    fingers = []
    # Thumb
    fingers.append(1 if hand_landmarks.landmark[tip_ids[0]].x < hand_landmarks.landmark[tip_ids[0]-1].x else 0)
    # Fingers
    for id in range(1, 5):
        fingers.append(1 if hand_landmarks.landmark[tip_ids[id]].y < hand_landmarks.landmark[tip_ids[id] - 2].y else 0)
    return fingers

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lm_list = []
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                lm_list.append((int(lm.x * w), int(lm.y * h)))

            if lm_list:
                x1, y1 = lm_list[8]  # Index tip
                x2, y2 = lm_list[12]  # Middle tip

                fingers = fingers_up(handLms)

                # Selection Mode: Two fingers up
                if fingers[1] and fingers[2]:
                    xp, yp = 0, 0
                    cv2.rectangle(img, (x1, y1-25), (x2, y2+25), draw_color, cv2.FILLED)
                    # Change color based on x1 position
                    if y1 < 100:
                        if 250 < x1 < 450:
                            draw_color = (255, 0, 255)  # Purple
                        elif 550 < x1 < 750:
                            draw_color = (0, 255, 0)    # Green
                        elif 800 < x1 < 950:
                            draw_color = (0, 0, 255)    # Red
                        elif 1000 < x1 < 1200:
                            draw_color = (0, 0, 0)      # Eraser

                # Drawing Mode: Index finger up
                elif fingers[1] and not fingers[2]:
                    cv2.circle(img, (x1, y1), 15, draw_color, cv2.FILLED)
                    if xp == 0 and yp == 0:
                        xp, yp = x1, y1
                    if draw_color == (0, 0, 0):
                        cv2.line(img, (xp, yp), (x1, y1), draw_color, eraser_thickness)
                        cv2.line(img_canvas, (xp, yp), (x1, y1), draw_color, eraser_thickness)
                    else:
                        cv2.line(img, (xp, yp), (x1, y1), draw_color, brush_thickness)
                        cv2.line(img_canvas, (xp, yp), (x1, y1), draw_color, brush_thickness)
                    xp, yp = x1, y1

            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

    # Merge canvas and webcam feed
    img_gray = cv2.cvtColor(img_canvas, cv2.COLOR_BGR2GRAY)
    _, img_inv = cv2.threshold(img_gray, 20, 255, cv2.THRESH_BINARY_INV)
    img_inv = cv2.cvtColor(img_inv, cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img, img_inv)
    img = cv2.bitwise_or(img, img_canvas)

    # Toolbar
    cv2.rectangle(img, (250, 0), (450, 100), (255, 0, 255), cv2.FILLED)
    cv2.rectangle(img, (550, 0), (750, 100), (0, 255, 0), cv2.FILLED)
    cv2.rectangle(img, (800, 0), (950, 100), (0, 0, 255), cv2.FILLED)
    cv2.rectangle(img, (1000, 0), (1200, 100), (0, 0, 0), cv2.FILLED)

    cv2.putText(img, "Save: Press 's'", (10, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Air Writing", img)

    key = cv2.waitKey(1)
    if key == ord('s'):
        timestamp = int(time.time())
        cv2.imwrite(f"air_drawing_{timestamp}.png", img_canvas)
        print("Artwork saved!")
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()


<h2 style='text-align:center;'>✅ Summary & Next Steps</h2>

You can extend these demos with brush size controls, saving strokes as SVG, or integrating with a Streamlit UI for web demos.